In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

# Import Pyod and the methods

In [2]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

C:\Users\Komal\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


# Import Metrics Package

In [3]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [4]:
mat_file_list=['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']

In [5]:
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

# Loading mat file

In [6]:
data=loadmat("mat_files/cardio.mat")


In [7]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [8]:
len(data)

5

In [9]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [10]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

In [11]:
df_columns=['Data','#Sample','#Dimensions','Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS','ABOD','IFOREST','FEATUREBAGGING']

# Precision Time and Roc evolution tables creation 

In [12]:
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


## Input(Independent) Feature shape in mat file format

In [13]:
type(data['X']),data['X'].shape

(numpy.ndarray, (1831, 21))

##  Dependent/ Target/ Output Feature shape in mat file format

In [14]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

# Exploring Mat files

In [15]:
from time import time
random_state = np.random.RandomState(42)

for mat_file in mat_file_list:
   print("\n... Processing", mat_file, '...')
   mat = loadmat(os.path.join('mat_files', mat_file))

   X = mat['X']
   y = mat['y'].ravel()
   outliers_fraction = np.count_nonzero(y) / len(y)
   outliers_percentage = round(outliers_fraction * 100, ndigits=4)

   # construct containers for saving results
   roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
   prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
   time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

   # 60% data for training and 40% for testing
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                       random_state=random_state)

   # standardizing data for processing
   X_train_norm, X_test_norm = standardizer(X_train, X_test)

   classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
       contamination=outliers_fraction),
       'Cluster-based Local Outlier Factor': CBLOF(
           contamination=outliers_fraction, check_estimator=False,
           random_state=random_state),
       'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                         random_state=random_state),
       'Histogram-base Outlier Detection (HBOS)': HBOS(
           contamination=outliers_fraction),
       'Isolation Forest': IForest(contamination=outliers_fraction,
                                   random_state=random_state),
       'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
       'Local Outlier Factor (LOF)': LOF(
           contamination=outliers_fraction),
       'Minimum Covariance Determinant (MCD)': MCD(
           contamination=outliers_fraction, random_state=random_state),
       'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
       'Principal Component Analysis (PCA)': PCA(
           contamination=outliers_fraction, random_state=random_state),
   }

   for clf_name, clf in classifiers.items():
       t0 = time()
       clf.fit(X_train_norm)
       test_scores = clf.decision_function(X_test_norm)
       t1 = time()
       duration = round(t1 - t0, ndigits=4)
       time_list.append(duration)

       roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
       prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

       print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
             'execution time: {duration}s'.format(
           clf_name=clf_name, roc=roc, prn=prn, duration=duration))

       roc_list.append(roc)
       prn_list.append(prn)

   temp_df = pd.DataFrame(time_list).transpose()
   temp_df.columns = df_columns
   time_df = pd.concat([time_df, temp_df], axis=0)

   temp_df = pd.DataFrame(roc_list).transpose()
   temp_df.columns = df_columns
   roc_df = pd.concat([roc_df, temp_df], axis=0)

   temp_df = pd.DataFrame(prn_list).transpose()
   temp_df.columns = df_columns
   prn_df = pd.concat([prn_df, temp_df], axis=0)




... Processing arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571, execution time: 1.3798s
Cluster-based Local Outlier Factor ROC:0.7684, precision @ rank n:0.4643, execution time: 1.6667s
Feature Bagging ROC:0.7799, precision @ rank n:0.5, execution time: 0.5834s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714, execution time: 1.3975s
Isolation Forest ROC:0.8527, precision @ rank n:0.5714, execution time: 0.3865s
K Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5, execution time: 0.0878s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643, execution time: 0.0758s


C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286, execution time: 0.6717s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5, execution time: 0.0565s
Principal Component Analysis (PCA) ROC:0.7997, precision @ rank n:0.5, execution time: 0.0688s

... Processing cardio.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.5763, precision @ rank n:0.1875, execution time: 0.4318s
Cluster-based Local Outlier Factor ROC:0.8221, precision @ rank n:0.4844, execution time: 0.1666s
Feature Bagging ROC:0.4879, precision @ rank n:0.1406, execution time: 0.7859s
Histogram-base Outlier Detection (HBOS) ROC:0.8453, precision @ rank n:0.4688, execution time: 0.006s
Isolation Forest ROC:0.9414, precision @ rank n:0.5, execution time: 0.4075s
K Nearest Neighbors (KNN) ROC:0.6959, precision @ rank n:0.2812, execution time: 0.1591s
Local Outlier Factor (LOF) ROC:0.4715, precision @ rank n:0.125, execution time: 0.0918s


C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-152.770956835748848 > -250.696836140088948). You may want to try with a higher value of support_fraction (current value: 0.511).
  RuntimeWarning)


Minimum Covariance Determinant (MCD) ROC:0.8778, precision @ rank n:0.3906, execution time: 0.4987s
One-class SVM (OCSVM) ROC:0.9507, precision @ rank n:0.5938, execution time: 0.1007s
Principal Component Analysis (PCA) ROC:0.9638, precision @ rank n:0.6875, execution time: 0.004s

... Processing glass.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7104, precision @ rank n:0.25, execution time: 0.0489s
Cluster-based Local Outlier Factor ROC:0.8506, precision @ rank n:0.25, execution time: 0.0539s
Feature Bagging ROC:0.7043, precision @ rank n:0.25, execution time: 0.0529s
Histogram-base Outlier Detection (HBOS) ROC:0.6524, precision @ rank n:0.0, execution time: 0.003s
Isolation Forest ROC:0.7195, precision @ rank n:0.25, execution time: 0.3202s
K Nearest Neighbors (KNN) ROC:0.7805, precision @ rank n:0.25, execution time: 0.01s
Local Outlier Factor (LOF) ROC:0.7774, precision @ rank n:0.25, execution time: 0.003s
Minimum Covariance Determinant (MCD) ROC:0.7165, precision @ rank n:0

...

Minimum Covariance Determinant (MCD) ROC:0.9903, precision @ rank n:0.7534, execution time: 10.78s
One-class SVM (OCSVM) ROC:0.9922, precision @ rank n:0.9553, execution time: 39.265s
Principal Component Analysis (PCA) ROC:0.9902, precision @ rank n:0.9503, execution time: 0.0259s

... Processing shuttle.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.6227, precision @ rank n:0.1938, execution time: 14.2932s
Cluster-based Local Outlier Factor ROC:0.6578, precision @ rank n:0.2206, execution time: 1.0278s
Feature Bagging ROC:0.5023, precision @ rank n:0.0669, execution time: 49.2008s
Histogram-base Outlier Detection (HBOS) ROC:0.9868, precision @ rank n:0.9386, execution time: 0.013s
Isolation Forest ROC:0.9974, precision @ rank n:0.9486, execution time: 2.4205s
K Nearest Neighbors (KNN) ROC:0.6549, precision @ rank n:0.2213, execution time: 7.4483s
Local Outlier Factor (LOF) ROC:0.5336, precision @ rank n:0.155, execution time: 10.0081s


C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.425442543000088 > -74.297176435073879). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.564705869881607 > -75.449647848888659). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.052965351125650 > -77.873043502795170). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\U

C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.493363258255101 > -75.585858300834488). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.872791578404886 > -81.239909999570003). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.133797032016744 > -76.364589262170597). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\U

C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.835499092286717 > -74.606867107292089). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.863734370680120 > -75.395146477137999). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Komal\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.807030513252769 > -76.759853309338808). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\U

Minimum Covariance Determinant (MCD) ROC:0.9897, precision @ rank n:0.7477, execution time: 10.3174s
One-class SVM (OCSVM) ROC:0.9923, precision @ rank n:0.9605, execution time: 39.0932s
Principal Component Analysis (PCA) ROC:0.9907, precision @ rank n:0.9577, execution time: 0.0249s

... Processing vertebral.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.3688, precision @ rank n:0.0, execution time: 0.0399s
Cluster-based Local Outlier Factor ROC:0.3891, precision @ rank n:0.0, execution time: 0.0489s
Feature Bagging ROC:0.3688, precision @ rank n:0.0, execution time: 0.0349s
Histogram-base Outlier Detection (HBOS) ROC:0.3617, precision @ rank n:0.0, execution time: 0.003s
Isolation Forest ROC:0.3539, precision @ rank n:0.0, execution time: 0.2633s
K Nearest Neighbors (KNN) ROC:0.3562, precision @ rank n:0.0, execution time: 0.0085s
Local Outlier Factor (LOF) ROC:0.3562, precision @ rank n:0.0, execution time: 0.002s
Minimum Covariance Determinant (MCD) ROC:0.4203, precision @ rank 

In [16]:
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1
0,optdigits,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5764,0.3824,0.4584,0.3486,0.4972,0.504
0,pendigits,6870,16,2.2707,0.7008,0.9609,0.4687,0.9294,0.9422,0.7602,0.481,0.8271,0.93,0.9332


In [17]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.2979,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9333,0.2333,0.1333,0.9667,1,1
0,optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,pendigits,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2769,0.0462,0.0462,0.0615,0.2923,0.3385


In [18]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,1.3798,1.6667,0.5834,1.3975,0.3865,0.0878,0.0758,0.6717,0.0565,0.0688
0,cardio,1831,21,9.6122,0.4318,0.1666,0.7859,0.006,0.4075,0.1591,0.0918,0.4987,0.1007,0.004
0,glass,214,9,4.2056,0.0489,0.0539,0.0529,0.003,0.3202,0.01,0.003,0.0389,0.001,0.002
0,ionosphere,351,33,35.8974,0.0873,0.0628,0.0958,0.009,0.3663,0.021,0.007,0.0838,0.0065,0.0027
0,letter,1600,32,6.25,0.4488,0.1746,0.9214,0.009,0.3848,0.1267,0.0943,1.1522,0.0997,0.006
0,lympho,148,18,4.0541,0.0399,0.0499,0.0479,0.0064,0.3211,0.006,0.002,0.033,0.001,0.001
0,mnist,7603,100,9.2069,6.2736,1.1256,58.3333,0.0758,1.7103,6.5167,6.1245,2.6378,4.1369,0.1106
0,musk,3062,166,3.1679,1.8516,0.3501,13.6278,0.0549,0.9734,1.716,1.6588,10.0665,1.1362,0.1137
0,optdigits,5216,64,2.8758,2.416,0.379,15.1759,0.0239,0.7164,1.6392,1.3988,1.1096,1.4536,0.0549
0,pendigits,6870,16,2.2707,1.5171,0.2479,3.9642,0.009,0.5545,0.4837,0.4672,1.7399,0.8088,0.006
